In [1]:
import numpy as np
import pandas as pd

In [4]:
credits= pd.read_csv('../Datos/credits.csv')
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [5]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [6]:
credits[credits.duplicated(keep=False)].sort_values(by='id') 
credits.drop_duplicates(inplace=True)

In [7]:
credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [8]:
credits.reset_index(drop=True, inplace=True)

In [9]:
#con ast convierto las cadenas de string a diccionarios
import ast

credits["cast"] = credits["cast"].apply(lambda x: ast.literal_eval(x) if not pd.isnull(x) else np.nan)
credits["crew"] = credits["crew"].apply(lambda x: ast.literal_eval(x) if not pd.isnull(x) else np.nan)

In [10]:
#ahora itero todo, guardo cada df en la lista que luego concatenare al nuevo df
datos = []
casts = credits["cast"]
id_original = credits["id"]

for idx, cast in enumerate(casts): 
    for d in cast:
        d["id"] = id_original[idx] # Agrego la columna "id" con el ID de la película
    df = pd.DataFrame.from_records(cast)
    datos.append(df)

cast_df = pd.concat(datos, ignore_index=True)


In [11]:
#crew
datos_crew= []
crews= credits["crew"]
id_original = credits["id"]

for idx, crew in enumerate(crews):
    for d in crew:
       d["id"] = id_original[idx] 
    df = pd.DataFrame.from_records(crew)
    datos_crew.append(df)
    
crew_df = pd.concat(datos_crew, ignore_index=True)

Con el fin de preparar los dataset, para las funciones 5 y 6. Realizare el merge corresponiente con el df de peliculas.

In [12]:
df_movies= pd.read_csv('movies_limpio.csv')
from pandas.core.reshape.merge import merge
#me da un error en el tipo de datos, por lo que convertire la columna id a numerico
df_movies['id'] = pd.to_numeric(df_movies['id'], errors='coerce')# los que no son numericos los paso a nan
df_movies = df_movies.dropna(subset=['id'])#aqui elimino los nan
df_movies['id'] = df_movies['id'].astype(int)#convierto a entero


C:\Users\clari\AppData\Local\Temp\ipykernel_1896\3333128663.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies= pd.read_csv('movies_limpio.csv')


In [17]:
actor = cast_df.merge(df_movies[['title', "return", 'id']], how="left", on="id")  

In [19]:
#elimino columnas innecesarias
actor.drop(columns=['cast_id','credit_id','gender','order','profile_path'], inplace=True)

In [20]:
actor

,character,id,name,title,return
0,Woody (voice),862,Tom Hanks,Toy Story,12.451801
1,Buzz Lightyear (voice),862,Tim Allen,Toy Story,12.451801
2,Mr. Potato Head (voice),862,Don Rickles,Toy Story,12.451801
3,Slinky Dog (voice),862,Jim Varney,Toy Story,12.451801
4,Rex (voice),862,Wallace Shawn,Toy Story,12.451801
...,...,...,...,...,...
562524,,227506,Iwan Mosschuchin,NaN,NaN
562525,,227506,Nathalie Lissenko,NaN,NaN
562526,,227506,Pavel Pavlov,NaN,NaN
562527,,227506,Aleksandr Chabrov,NaN,NaN


In [21]:
director = crew_df.merge(df_movies[['title', 'release_date','budget', 'revenue','return', 'id']], how='left', on='id')  
director.drop(columns=['credit_id','department','gender','profile_path'], inplace=True)
director

,id,job,name,title,release_date,budget,revenue,return
0,862,Director,John Lasseter,Toy Story,1995-10-30,30000000,373554033.0,12.451801
1,862,Screenplay,Joss Whedon,Toy Story,1995-10-30,30000000,373554033.0,12.451801
2,862,Screenplay,Andrew Stanton,Toy Story,1995-10-30,30000000,373554033.0,12.451801
3,862,Screenplay,Joel Cohen,Toy Story,1995-10-30,30000000,373554033.0,12.451801
4,862,Screenplay,Alec Sokolow,Toy Story,1995-10-30,30000000,373554033.0,12.451801
...,...,...,...,...,...,...,...,...
464358,67758,Original Music Composer,Richard McHugh,NaN,NaN,NaN,NaN,NaN
464359,67758,Director of Photography,João Fernandes,NaN,NaN,NaN,NaN,NaN
464360,227506,Director,Yakov Protazanov,NaN,NaN,NaN,NaN,NaN
464361,227506,Producer,Joseph N. Ermolieff,NaN,NaN,NaN,NaN,NaN


In [23]:
#exporto los df creados a csv (elimine las columnas innecesarias ya que render solo nos da 512mb)
director.to_csv('director.csv')
actor.to_csv('actor.csv')